## instructions

got data from churn.csv

dropped column of ids

onehotencoded the categorical fields

normalized the non categorical (money stuff and tenure)

remove nans

get labels from Churn column

drop the churn column

now you can train


https://www.youtube.com/watch?v=bqhQ2LWBheQ - normalizing

In [1557]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils as torch_utils
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [1449]:
from datasets import load_dataset

dataset = load_dataset("scikit-learn/churn-prediction", split="train").to_csv("churn.csv")

Creating CSV from Arrow format: 100%|██████████| 8/8 [00:00<00:00, 276.25ba/s]


Customer churn prediction dataset of a fictional telecommunication company made by IBM Sample Datasets. Context Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs.

Content Each row represents a customer, each column contains customer’s attributes described on the column metadata.

The data set includes information about:

Customers who left within the last month: the column is called Churn
Services that each customer has signed up for: phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
Customer account information: how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
Demographic info about customers: gender, age range, and if they have partners and dependents

In [1450]:
data = pd.read_csv("churn.csv")
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [1451]:
data.drop(columns=["customerID"], axis="columns", inplace=True)

print(data.columns)
print(len(data.columns))

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')
20


In [1452]:
data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [1453]:
uniqGender = data['gender'].unique()
uniqSeniorCitizen = data['SeniorCitizen'].unique()
uniqPartner = data['Partner'].unique()
uniqDependents = data['Dependents'].unique()
# uniqTenure = data['tenure'].unique() # ! TENURE IS NOT CATEGORY
uniqPhoneService = data['PhoneService'].unique()
uniqMultipleLines = data['MultipleLines'].unique()
uniqInternetService = data['InternetService'].unique()
uniqOnlineSecurity = data['OnlineSecurity'].unique()
uniqOnlineBackup = data['OnlineBackup'].unique()
uniqDeviceProtection = data['DeviceProtection'].unique()
uniqTechSupport = data['TechSupport'].unique()
uniqStreamingTV = data['StreamingTV'].unique()
uniqStreamingMovies = data['StreamingMovies'].unique()
uniqContract = data['Contract'].unique()
uniqPaperlessBilling = data['PaperlessBilling'].unique()
uniqPaymentMethod = data['PaymentMethod'].unique()
# uniqMonthlyCharges = data['MonthlyCharges'].unique()  # ! IS NOT CATEGORY
# uniqTotalCharges = data['TotalCharges'].unique() # ! IS NOT CATEGORY
uniqChurn = data['Churn'].unique()

In [1454]:
data.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [1455]:
one_hot_encoded = pd.get_dummies(data, columns=[
    'gender', 'SeniorCitizen', 'Partner', 'Dependents',
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'], dtype=int)

# DONT RE RUN HERE

In [1456]:
data = one_hot_encoded

data.to_csv("checkpoint.csv")

In [1491]:
data = pd.read_csv('checkpoint.csv')

In [1492]:
data

,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,29.85,No,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
1,34,56.95,1889.50,No,0,1,1,0,1,0,...,0,0,1,0,1,0,0,0,0,1
2,2,53.85,108.15,Yes,0,1,1,0,1,0,...,0,1,0,0,0,1,0,0,0,1
3,45,42.30,1840.75,No,0,1,1,0,1,0,...,0,0,1,0,1,0,1,0,0,0
4,2,70.70,151.65,Yes,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,24,84.80,1990.50,No,0,1,1,0,0,1,...,1,0,1,0,0,1,0,0,0,1
7039,72,103.20,7362.90,No,1,0,1,0,0,1,...,1,0,1,0,0,1,0,1,0,0
7040,11,29.60,346.45,No,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
7041,4,74.40,306.60,Yes,0,1,0,1,0,1,...,0,1,0,0,0,1,0,0,0,1


In [1493]:
data.isna().values.any()

nan_indices = data.isna().stack()[lambda x: x].index.tolist()

# Print indices of NaN values
for index in nan_indices:
    print(f"NaN value at index {index}: {data.loc[index]}")

data = data.dropna()

NaN value at index (488, 'TotalCharges'): nan
NaN value at index (753, 'TotalCharges'): nan
NaN value at index (936, 'TotalCharges'): nan
NaN value at index (1082, 'TotalCharges'): nan
NaN value at index (1340, 'TotalCharges'): nan
NaN value at index (3331, 'TotalCharges'): nan
NaN value at index (3826, 'TotalCharges'): nan
NaN value at index (4380, 'TotalCharges'): nan
NaN value at index (5218, 'TotalCharges'): nan
NaN value at index (6670, 'TotalCharges'): nan
NaN value at index (6754, 'TotalCharges'): nan


In [1495]:
labels = LabelEncoder().fit_transform(data['Churn'])
print(labels)
print(len(labels))

[0 0 1 ... 0 1 0]
7032


# need to normalize tenure, monthly charges, total charges

In [1496]:
data.drop(columns=["Churn"], axis="columns", inplace=True)

C:\Users\johns\AppData\Local\Temp\ipykernel_7188\4089256921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=["Churn"], axis="columns", inplace=True)


In [1497]:
data

,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,29.85,1,0,1,0,0,1,1,...,0,1,0,0,0,1,0,0,1,0
1,34,56.95,1889.50,0,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,1
2,2,53.85,108.15,0,1,1,0,1,0,1,...,0,1,0,0,0,1,0,0,0,1
3,45,42.30,1840.75,0,1,1,0,1,0,1,...,0,0,1,0,1,0,1,0,0,0
4,2,70.70,151.65,1,0,1,0,1,0,1,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,24,84.80,1990.50,0,1,1,0,0,1,0,...,1,0,1,0,0,1,0,0,0,1
7039,72,103.20,7362.90,1,0,1,0,0,1,0,...,1,0,1,0,0,1,0,1,0,0
7040,11,29.60,346.45,1,0,1,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
7041,4,74.40,306.60,0,1,0,1,0,1,1,...,0,1,0,0,0,1,0,0,0,1


In [1498]:
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [1500]:
data

array([[0.        , 0.11542289, 0.0012751 , ..., 0.        , 1.        ,
        0.        ],
       [0.46478873, 0.38507463, 0.21586661, ..., 0.        , 0.        ,
        1.        ],
       [0.01408451, 0.35422886, 0.01031041, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.14084507, 0.11293532, 0.03780868, ..., 0.        , 1.        ,
        0.        ],
       [0.04225352, 0.55870647, 0.03321025, ..., 0.        , 0.        ,
        1.        ],
       [0.91549296, 0.86965174, 0.78764136, ..., 0.        , 0.        ,
        0.        ]])

In [1501]:
torch.save(data, "tensors.pt")

In [1502]:
loaded_tensor = torch.load('tensors.pt')

In [1503]:
loaded_tensor

array([[0.        , 0.11542289, 0.0012751 , ..., 0.        , 1.        ,
        0.        ],
       [0.46478873, 0.38507463, 0.21586661, ..., 0.        , 0.        ,
        1.        ],
       [0.01408451, 0.35422886, 0.01031041, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.14084507, 0.11293532, 0.03780868, ..., 0.        , 1.        ,
        0.        ],
       [0.04225352, 0.55870647, 0.03321025, ..., 0.        , 0.        ,
        1.        ],
       [0.91549296, 0.86965174, 0.78764136, ..., 0.        , 0.        ,
        0.        ]])

## data is ready now

In [1504]:
loaded_tensor.shape

(7032, 46)

In [1505]:
X = torch.tensor(data)
X

tensor([[0.0000, 0.1154, 0.0013,  ..., 0.0000, 1.0000, 0.0000],
        [0.4648, 0.3851, 0.2159,  ..., 0.0000, 0.0000, 1.0000],
        [0.0141, 0.3542, 0.0103,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.1408, 0.1129, 0.0378,  ..., 0.0000, 1.0000, 0.0000],
        [0.0423, 0.5587, 0.0332,  ..., 0.0000, 0.0000, 1.0000],
        [0.9155, 0.8697, 0.7876,  ..., 0.0000, 0.0000, 0.0000]],
       dtype=torch.float64)

In [1506]:
class ChurnModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.l1 = nn.Linear(in_features=46, out_features=7)
        self.l2 = nn.Linear(in_features=7, out_features=1)

    def forward(self, x):
        return self.l2(self.l1(x))

In [1507]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [1508]:
torch.manual_seed(7)
ChurnModel_Inst1 = ChurnModel().to(device)

ChurnModel_Inst1

ChurnModel(
  (l1): Linear(in_features=46, out_features=7, bias=True)
  (l2): Linear(in_features=7, out_features=1, bias=True)
)

In [1509]:
ChurnModel_Inst1.eval()
with torch.inference_mode():
    untrained = ChurnModel_Inst1(X.to(device=device, dtype=torch.float32))

print(untrained)
print(len(untrained))
# made 7043 predictions based on the data provided

tensor([[-0.3646],
        [-0.2963],
        [-0.1563],
        ...,
        [-0.5314],
        [-0.2373],
        [-0.0391]], device='cuda:0')
7032


In [1510]:
loss_fn = nn.BCEWithLogitsLoss() # using this instead of BCELoss to put the sigmoid activation function (adds non-linearity to the model for better learning) in one operation
optimizer_adam = optim.Adam(ChurnModel_Inst1.parameters(), lr=0.001)  # Adam optimizer # ! 
optimizer_sgd = optim.SGD(ChurnModel_Inst1.parameters(), lr=0.01)  # SGD optimizer with momentum # !
optimizer_rmsprop = optim.RMSprop(ChurnModel_Inst1.parameters(), lr=0.001)  # RMSprop optimizer # ! 
optimizer_adagrad = optim.Adagrad(ChurnModel_Inst1.parameters(), lr=0.01)  # Adagrad optimizer # ! 
optimizer_adamw = optim.AdamW(ChurnModel_Inst1.parameters(), lr=0.001)  # AdamW optimizer # ! 
optimizer_radam = optim.RAdam(ChurnModel_Inst1.parameters(), lr=0.001)  # RAdam optimizer # ! 

In [1511]:
optimizer = optimizer_sgd

In [1512]:
tensor_labels = torch.tensor(labels, dtype=torch.float64)
tensor_labels

tensor([0., 0., 1.,  ..., 0., 1., 0.], dtype=torch.float64)

In [1513]:
# SPLIT THE DATA

X_train, X_test, y_train, y_test = train_test_split(X, tensor_labels, test_size=0.2, random_state=7)

X_train = X_train.to(device=device, dtype=torch.float32)
X_test = X_test.to(device=device, dtype=torch.float32)
y_train = y_train.to(device=device, dtype=torch.float32)
y_test = y_test.to(device=device, dtype=torch.float32)
len(X_train), len(X_test), len(y_train), len(y_test)

(5625, 1407, 5625, 1407)

In [1514]:
X_train.dtype

torch.float32

In [1515]:
# get the accuracy of it
testTens=torch.arange(1,11,step=1)
testTens2=torch.arange(1,20,step=2)

print(testTens)
print(testTens2)

def accuracy_fn(truth, pred):
    correct = torch.eq(truth, pred).sum().item()
    percentage = correct / len(pred) * 100
    print(f"accuracy: {percentage} %")

accuracy_fn(testTens, testTens2)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
tensor([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19])
accuracy: 10.0 %


EXPLODING GRADIENTS ISSUE IS HERE I THINK

In [1551]:
epochs = 100

In [1556]:
for epoch in range(epochs):

    print(epoch)

    ChurnModel_Inst1.train()

    out_logits = ChurnModel_Inst1(X_train).squeeze()

    out_preds = torch.round(torch.sigmoid(out_logits))

    print(out_logits)
    print(out_preds)

    loss = loss_fn(out_logits, y_train)

    print(f"train loss: {loss}")

    accuracy = accuracy_fn(out_preds, y_train)

    optimizer.zero_grad()

    loss.backward()

    #! sdgeg

    optimizer.step()
    
    ChurnModel_Inst1.eval()
    with torch.inference_mode():
        test_out_logits = ChurnModel_Inst1(X_test.to(device=device, dtype=torch.float32)).squeeze()
        test_out_preds = torch.round(torch.sigmoid(test_out_logits))

        print(test_out_logits)
        print(test_out_preds)

        loss = loss_fn(test_out_logits, y_test)

        print(f"test loss: {loss}")

        accuracy = accuracy_fn(test_out_preds, y_test)

0
tensor([-1.6739, -0.7328, -0.2295,  ..., -0.8877, -0.7652, -0.6672],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<RoundBackward0>)
train loss: 0.45512738823890686
accuracy: 77.72444444444444 %
tensor([-1.6823, -0.9780,  0.1428,  ..., -0.5904,  0.2597, -1.7295],
       device='cuda:0')
tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:0')
test loss: 0.46465522050857544
accuracy: 76.54584221748401 %
1
tensor([-1.6753, -0.7327, -0.2285,  ..., -0.8882, -0.7648, -0.6674],
       device='cuda:0', grad_fn=<SqueezeBackward0>)
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<RoundBackward0>)
train loss: 0.45503732562065125
accuracy: 77.77777777777779 %
tensor([-1.6839, -0.9781,  0.1442,  ..., -0.5904,  0.2611, -1.7310],
       device='cuda:0')
tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:0')
test loss: 0.46455883979797363
accuracy: 76.54584221748401 %
2
tensor([-1.6767, -0.7326, -0.2

## WE GETTING HIGHER ACCURACY LETS FUCKING TGOOOOOOOO

just need to figure out why its getting nan after the first epoch...

In [1487]:
# save model

from pathlib import Path

path = Path('saved_models')    

def get_latest_file():
    
    filelist = list(path.iterdir())

    filelistnums = []

    for file in filelist:

        filename = str(file).split("_")

        print(filename[2])

        filename = filename[2].split('.')

        print(f"filenum: {filename[0]}")

        filelistnums.append(int(filename[0]))
        

    return max(filelistnums)


def save_model(model2save, pathname, modelpathname):

    namenum = '_' + str(get_latest_file() + 1)

    MODEL_PATH = Path(pathname)
    MODEL_PATH.mkdir(parents=True, exist_ok=True)
    MODEL_NAME = f"{modelpathname + namenum}.pt"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    print(f"saving {MODEL_NAME}")
    torch.save(obj=model2save.state_dict(), f=MODEL_SAVE_PATH)

In [1488]:
save_model(ChurnModel_Inst1, pathname="saved_models", modelpathname="best")

1.pt
filenum: 1
2.pt
filenum: 2
3.pt
filenum: 3
4.pt
filenum: 4
1
filenum: 1
saving best_5.pt


In [1489]:
ChurnModel_Inst2 = ChurnModel().to(device)

ChurnModel_Inst2.load_state_dict(torch.load(f="saved_models/best1.pt"))

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/best1.pt'

In [1559]:
ChurnModel_Inst2.eval()
with torch.inference_mode():
        b = ChurnModel_Inst1(X_test.to(device=device, dtype=torch.float32)).squeeze()
        c = torch.round(torch.sigmoid(b))

        print(b)
        print(c)

        bloss = loss_fn(b, y_test)

        print(bloss)

        accuracy = accuracy_fn(c, y_test)

tensor([-1.8294, -0.9924,  0.2589,  ..., -0.5996,  0.3840, -1.8628],
       device='cuda:0')
tensor([0., 0., 1.,  ..., 0., 1., 0.], device='cuda:0')
tensor(0.4566, device='cuda:0')
accuracy: 77.68301350390902 %
